In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from haystack_argilla.base import ArgillaCallback
import argilla as rg
from datasets import load_dataset

from haystack.agents import Tool
from haystack.agents.memory import ConversationSummaryMemory
from haystack.agents import AgentStep, Agent
from haystack.agents.base import Agent, ToolsManager

from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser, BM25Retriever
from haystack.pipelines import Pipeline
from haystack.utils import print_answers

Importing base


In [ ]:
#Create basic DS

ds = rg.FeedbackDataset.for_retrieval_augmented_generation(1)
records = rg.FeedbackRecord(
    fields={
        "query": "Why can camels survive long without water?",
        "retrieved_document_1": "because they are camels"
    },
)
ds.add_records(records)
ds.push_to_argilla("haystack_argilla")

In [3]:
#OPEN AI KEY
openai_api_key = ...

In [ ]:
# HAYSTACK 
# based on tutorial https://haystack.deepset.ai/tutorials/25_customizing_agent

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")

## CREATE GENERATIVE PIPELINE
document_store = InMemoryDocumentStore(use_bm25=True)
document_store.write_documents(dataset)
retriever = BM25Retriever(document_store=document_store)
prompt0 = "Please use a maximum of 50 tokens. Question: {query}\nDocuments: {join(documents)}\nAnswer:"
prompt_template = PromptTemplate(
    prompt=prompt0,
    output_parser=AnswerParser(),
)
prompt_node = PromptNode(
    model_name_or_path="gpt-3.5-turbo-instruct", api_key=openai_api_key, default_prompt_template=prompt_template
)
generative_pipeline = Pipeline()
generative_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
generative_pipeline.add_node(component=prompt_node, name="Prompt", inputs=["Retriever"])

## CREATE AGENT
search_tool = Tool(
    name="Search_the_documents_tool",
    pipeline_or_node=generative_pipeline,
    description="useful for when you need to answer questions about the seven wonders of the world",
    output_variable="answers",
)

search_tool.run("Who lived in a town in Lydia?")

agent_prompt_node = PromptNode(
    "gpt-3.5-turbo",
    api_key=openai_api_key,
    max_length=256,
    stop_words=["Observation:"],
    model_kwargs={"temperature": 0.5},
)
memory_prompt_node = PromptNode(
    model_name_or_path="philschmid/bart-large-cnn-samsum", max_length=256, model_kwargs={"task_name":"text2text-generation"}  # MODEL
)
memory = ConversationSummaryMemory(
    prompt_node=memory_prompt_node, prompt_template="{chat_transcript}"
)
agent_prompt = """
In the following conversation, a human user interacts with an AI Agent. The human user poses questions, and the AI Agent goes through several steps to provide well-informed answers.
The AI Agent must use the available tools to find the up-to-date information. The final answer to the question should be truthfully based solely on the output of the tools. The AI Agent should ignore its knowledge when answering the questions.
The AI Agent has access to these tools:
{tool_names_with_descriptions}

The following is the previous conversation between a human and The AI Agent:
{memory}

AI Agent responses must start with one of the following:

Thought: [the AI Agent's reasoning process]
Tool: [tool names] (on a new line) Tool Input: [input as a question for the selected tool WITHOUT quotation marks and on a new line] (These must always be provided together and on separate lines.)
Observation: [tool's result]
Final Answer: [final answer to the human user's question]

When selecting a tool, the AI Agent must provide both the "Tool:" and "Tool Input:" pair in the same response, but on separate lines.

The AI Agent should not ask the human user for additional information, clarification, or context.
If the AI Agent cannot find a specific answer, it should accept the first word of the answer it found as the answer to the query.

Question: {query}
Thought:
{transcript}
"""

def resolver_function(query, agent, agent_step):
    return {
        "query": query,
        "tool_names_with_descriptions": agent.tm.get_tool_names_with_descriptions(),
        "transcript": agent_step.transcript,
        "memory": agent.memory.load(),
    }

In [5]:
# CREATE AGENT

conversational_agent = Agent(
    agent_prompt_node,
    prompt_template=agent_prompt,
    prompt_parameters_resolver=resolver_function,
    memory=memory,
    tools_manager=ToolsManager([search_tool]),
)

In [ ]:
# ARGILLA CALLBACK

api_key = "argilla.apikey"
api_url = "http://localhost:6900/"
dataset_id = "haystack_argilla"

ArgillaCallback(agent=conversational_agent, dataset_name=dataset_id, api_url=api_url, api_key=api_key)

In [9]:
conversational_agent.run(query="What is another name for Artemis?")

Importing base

Agent custom-at-query-time started with {'query': 'What is another name for Artemis?', 'params': None}
The human user is asking for another name for Artemis. Based on the previous information, the AI Agent knows that Artemis is also known as Diana in Roman mythology. To provide the answer, the AI Agent can use the 'Search_the_documents_tool' to confirm this information.

Tool: Search_the_documents_tool
Tool Input: What is another name for Artemis?

Observation: Diana
Thought: The 'Search_the_documents_tool' provided the answer 'Diana' as another name for Artemis.

Final Answer: Diana

/Users/kursat/anaconda3/envs/argilla/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/Users/kursat/anaconda3/envs/argilla/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:437: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


Output()

Records have been updated to Argilla


{'query': 'What is another name for Artemis?',
 'answers': [<Answer {'answer': 'Diana', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {}}>],
 'transcript': "The human user is asking for another name for Artemis. Based on the previous information, the AI Agent knows that Artemis is also known as Diana in Roman mythology. To provide the answer, the AI Agent can use the 'Search_the_documents_tool' to confirm this information.\n\nTool: Search_the_documents_tool\nTool Input: What is another name for Artemis?\nObservation: Diana\nThought:The 'Search_the_documents_tool' provided the answer 'Diana' as another name for Artemis.\n\nFinal Answer: Diana"}